In [1]:
import pandas as pd
import numpy as np
import spacy
nlp = spacy.load("en_core_web_sm")

# remove misformatted row due to bad html
raw_df = (
    pd.read_csv('data/raw/quakenu_raw.csv')
        .drop(16803, axis=0)
)

# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)
# print(raw_df[:1])
# print(raw_df.dtypes)

In [2]:
import datetime

cleaning_test_df = raw_df[::1000] # every 1000th post

def clean_post_df(raw_df):
    clean_df = raw_df
    clean_df['date'] = pd.to_datetime(clean_df['date'], format='%Y-%m-%d, %H:%M')
    return clean_df

clean_test_df = clean_post_df(cleaning_test_df)
clean_df = clean_post_df(raw_df)

C:\Users\BYRONA~1\AppData\Local\Temp/ipykernel_6756/1649249616.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['date'] = pd.to_datetime(clean_df['date'], format='%Y-%m-%d, %H:%M')


In [3]:
import re
raw_text_col = raw_df['post_text'].str.replace('[^\w\']', ' ', flags=re.UNICODE)

C:\Users\BYRONA~1\AppData\Local\Temp/ipykernel_6756/2140873254.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  raw_text_col = raw_df['post_text'].str.replace('[^\w\']', ' ', flags=re.UNICODE)


In [158]:
# print(raw_df.columns)
OLD_MEMBER_CUTOFF = 50
MIN_POST_CUTOFF = 5

min_post_df = clean_df.groupby('author').filter(lambda group : len(group) > MIN_POST_CUTOFF)

forum_first_post_dt = min_post_df['date'].min()
forum_time_range = min_post_df['date'].max() - forum_first_post_dt
# print(forum_time_range)

author_groups = min_post_df.groupby('author')
last_post_dt = author_groups['date'].max()
first_post_dt = author_groups['date'].min()
post_time_range = last_post_dt - first_post_dt
last_post_era = (last_post_dt - forum_first_post_dt) / forum_time_range

# print(last_post_era)

post_frequency_seconds = post_time_range.dt.total_seconds() / author_groups['date'].count()
median_post_freq = author_groups['date'].agg(
    lambda group: group.sort_values().diff().median()
).dt.total_seconds()
is_old_member = author_groups['date'].count() > OLD_MEMBER_CUTOFF

median_post_length_chars = min_post_df['post_text'].str.len().dropna().groupby(min_post_df['author']).median()

# print(sum(is_old_member))
# print(raw_df[raw_df['post_text'].isna()])

In [153]:
post_counts = author_groups['date'].agg(
    lambda group: group.count() # group.sort_values().diff().count()
)

sum(post_counts > 2)

print(median_post_freq.dt.total_seconds())

author
0        3.045e+06
1        1.524e+04
100      5.831e+06
1000     4.146e+04
10002    2.692e+06
           ...    
9978     3.657e+04
9981     8.256e+04
9993     1.800e+02
9995     1.230e+04
9996     8.148e+04
Name: date, Length: 1283, dtype: float64


In [159]:
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix 

# X = pd.concat([post_frequency_seconds, last_post_era], axis=1, keys=['post_freq', 'post_era'])
X = pd.concat([median_post_freq, last_post_era, median_post_length_chars], axis=1, keys=['post_freq', 'post_era', 'post_len'])
X['post_len'].fillna(X['post_len'].median(), inplace=True)
y = is_old_member

np.random.seed(3342)
X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
classifier = LogisticRegression()

old_member_pipeline = Pipeline([
    ('scaler', scaler),
    ('classifier', classifier)
])

In [160]:
old_member_pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', LogisticRegression())])

In [161]:
y_hat_test = old_member_pipeline.predict(X_test)

print(classification_report(y_test, y_hat_test))

y_hat_train = old_member_pipeline.predict(X_train)

print(classification_report(y_train, y_hat_train))

              precision    recall  f1-score   support

       False       0.79      0.90      0.84       154
        True       0.64      0.42      0.50        65

    accuracy                           0.76       219
   macro avg       0.71      0.66      0.67       219
weighted avg       0.74      0.76      0.74       219

              precision    recall  f1-score   support

       False       0.80      0.90      0.85       465
        True       0.65      0.45      0.53       190

    accuracy                           0.77       655
   macro avg       0.73      0.68      0.69       655
weighted avg       0.76      0.77      0.76       655



In [40]:
print(raw_df['post_text'].str.len().groupby(raw_df['author'], sort=False).mean())

author
3       270.997
122     106.881
6       154.400
1       305.435
47      184.000
         ...   
2519    347.333
3415    157.000
3533    414.000
3395     67.000
2533    835.500
Name: post_text, Length: 2105, dtype: float64


In [162]:
"""
Text gathering/cleaning issues
    - scraper not catching underlines, italics, etc https://www.quakeworld.nu/forum/topic/28
    - bb-code divs https://www.quakeworld.nu/forum/topic/28
    - 
"""

pd.set_option('display.max_colwidth', None)
"""
Cleaning a single string for NLP
    
    - remove nonchars except for apostrophes

"""

print(raw_df[['thread', 'post_id', 'post_text']].sample(n=5))
# print(raw_text_col.sample(n=5))
# print([text for text in raw_text_col.sample(n=15)])

       thread post_id  \
71009      52   13465   
12233    1603   81343   
40107    3717   46730   
78437     104   31865   
58465    6990  105734   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [167]:
import unidecode

test_accent_str = "Sí señor, ¿qué necesita?"

print(unidecode.unidecode(test_accent_str))

test_lemma_str = "i guess canadians aren't invited. i see how it is..."

test_lemma_doc = nlp(test_lemma_str)

for token in test_lemma_doc:
    print(token.lemma_)
    # print(token.is_sent_start)

# print(' '.join([token.lemma_ for token in test_lemma_doc]))
# print(test_lemma_doc.cats)

Si senor, ?que necesita?
I
guess
canadian
be
not
invite
.
I
see
how
it
be
...
